In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import chirp

In [2]:
def velfield(ts):
    
    path = 'C:/Users/bono2/Desktop/Pitching Airfoil/dataset/raw_chirp/klee_Chirp_BoA30_f0p05f0p5_L09000/ibpm'\
                + str(format(ts, '05'))+'.plt'
    with open(path) as file:
        f_text = file.read()

    f_list = f_text[191:]
    f_list = f_list.split(' ')
    f_list.pop()

    UX, UY = [], []
    for idx, val in enumerate(f_list):
        if val[0] == '\n':
            UX.append(float(f_list[idx+2]))
            UY.append(float(f_list[idx+3]))
    UX = np.array(UX); UY = np.array(UY)

    UX = UX.reshape(299, 599)
    UY = UY.reshape(299, 599)

    return UX, UY

## define ibpmtimestep <-> frequency

In [1]:
ibpm_time = np.arange(5510, 14500, 10)
freq = 0.05 + ((ibpm_time - 5501)/100)/90*(0.5-0.05)

ori_time = np.arange(0, 90, 0.01)
angle = chirp(ori_time, 0.05, 45, 0.5, method = 'linear', phi = 0)
angle = angle[9::10][:-1]

# ori_freq = 0.05 + ori_time/90*(0.5-0.05)
# ori_time = ori_time[9::10]

# plt.plot((ibpm_time - 5502)/100, freq)
# plt.plot(ori_time, ori_freq[9::10])
# plt.show()
# plt.plot(angle)

# print(len(ibpm_time),len(angle), ori_time[-1], (ibpm_time[-1]-5501)/100)

# print(angle[0], np.cos(2*np.pi*freq*(ibpm_time - 5501)/100)[0])

# plt.plot(ibpm_time, angle)
# plt.plot(ibpm_time, np.cos(2*np.pi*freq*(ibpm_time - 5501)/100))

NameError: name 'np' is not defined

## Create Dataset & Preprocess

In [5]:
chirp_X = np.zeros((len(ibpm_time), 200, 200))

for i, timestep in enumerate(ibpm_time):
    
    UX, UY = velfield(timestep)
    
    UX = UX[99:199, 399:599]
    UY = UY[99:199, 399:599]
    
    chirp_X[i] = np.concatenate((UX, UY))

In [6]:
chirp_X = chirp_X.reshape(len(ibpm_time), 200, 200,1)
chirp_X -= np.mean(chirp_X, axis = 0)

In [7]:
train_ux = chirp_X[:,0:100,:,:]; train_uy = chirp_X[:,100:200,:,:]

ux_max = 0; ux_min = 10000; uy_max = 0; uy_min = 10000

for ux in train_ux:
    M = np.max(ux); m = np.min(ux)
    if ux_max < M:
        ux_max = M
    elif ux_min > m:
        ux_min = m
   
for uy in train_uy:
    M = np.max(uy); m = np.min(uy)
    if uy_max < M:
        uy_max = M
    if uy_min > m:
        uy_min = m
        
print(ux_max, uy_max, ux_min, uy_min)
ux_range = ux_max - ux_min
uy_range = uy_max - uy_min

train_ux /= ux_range; train_uy /= uy_range

0.20043061957730868 0.4182108561579533 -0.2575932157953289 -0.40945621575750835


In [8]:
for x, y, i in zip(train_ux, train_uy, range(len(train_uy))):
    chirp_X[i,0:100,:,:] = x
    chirp_X[i,100:200,:,:] = y

In [9]:
np.save('C:/Users/bono2/Desktop/Pitching Airfoil/dataset/dataset_chirp/Chirp_BoA30_f0p05f0p5_L09000.npy', chirp_X)

In [10]:
np.save('C:/Users/bono2/Desktop/Pitching Airfoil/dataset/dataset_chirp/FREQ_Chirp_BoA30_f0p05f0p5_L09000.npy', freq)
np.save('C:/Users/bono2/Desktop/Pitching Airfoil/dataset/dataset_chirp/ANGLE_Chirp_BoA30_f0p05f0p5_L09000.npy', angle)

In [ ]:
def kin():
    # Loading .kin file
    path = 'C:/Users/bono2/Desktop/Pitching Airfoil/dataset/raw_chirp/klee_Chirp_BoA30_f0p05f0p5_L09000/ibpm.kin'
    with open(path) as file:
        f_text = file.read()

    f_text = f_text.replace('\n', ' ')
    f_list = f_text.split(' ')

    timestep = 0
    length = 0

    my_alpha, my_alphadot = [], []

    for idx, item in enumerate(f_list):
        if item == str(timestep):
            timestep += 1
            length += 1
            my_alpha.append(float(f_list[idx + 2]))
            my_alphadot.append(float(f_list[idx + 3]))

        else:
            pass
    return my_alpha, my_alphadot